<a href="https://colab.research.google.com/github/sajacaros/bootcamp_colab/blob/main/%5BML_Project%5D_%EC%A0%84%EB%A0%A5%EC%82%AC%EC%9A%A9%EB%9F%89_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기


데이터 : https://dacon.io/competitions/official/235736/overview/description

In [35]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

In [36]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [39]:
# flag setting
feature_reducing = False
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = mean_absolute_error

In [40]:
# 데이터를 불러옵니다.
file_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/energy/'
train_df = pd.read_csv(file_path+'train.csv', encoding='cp949')
test_df = pd.read_csv(file_path+'test.csv',  encoding='cp949')
submission_df = pd.read_csv(file_path+'sample_submission.csv',  encoding='cp949')

In [41]:
train = train_df.copy()
test = test_df.copy()

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [42]:
## On your Own
# 1. 결측치 확인 (v)
# 2. dtype이 object인 column --> categorical feature (v)
# 3. target value distribution
# train.info()
# train.info(memory_usage='deep')
# test.head(10)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   num           10080 non-null  int64  
 1   date_time     10080 non-null  object 
 2   기온(°C)        3360 non-null   float64
 3   풍속(m/s)       3360 non-null   float64
 4   습도(%)         3360 non-null   float64
 5   강수량(mm, 6시간)  1680 non-null   float64
 6   일조(hr, 3시간)   3360 non-null   float64
 7   비전기냉방설비운영     2296 non-null   float64
 8   태양광보유         1624 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 708.9+ KB


이런 식으로 여러가지 그래프를 그려가며, 데이터에 대한 인사이트를 얻습니다!

### 3. 전처리

#### 결측치 처리

In [43]:
train.columns = ['num', 'date_time', 'target', 'temperature', 'windspeed', 'humidity',
                 'precipitation', 'insolation', 'is_cool', 'is_solar']
test.columns = ['num', 'date_time', 'temperature', 'windspeed', 'humidity',
                 'precipitation', 'insolation', 'is_cool', 'is_solar']
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num            122400 non-null  int64  
 1   date_time      122400 non-null  object 
 2   target         122400 non-null  float64
 3   temperature    122400 non-null  float64
 4   windspeed      122400 non-null  float64
 5   humidity       122400 non-null  float64
 6   precipitation  122400 non-null  float64
 7   insolation     122400 non-null  float64
 8   is_cool        122400 non-null  float64
 9   is_solar       122400 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 9.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            10080 non-null  int64  
 1   date_time      10080 non-null  object 
 2 

In [46]:
# Q. 같은 건물이면 2020-06-01 ~ 2020-08-24 기간 동안 is_cool, is_solar 정보 확인
# 빌딩 정보 (is_cool, is_solar) - test 값 채워주기 위해
building_info = train[['num', 'is_cool', 'is_solar']].drop_duplicates()
test = test.drop(columns=['is_cool', 'is_solar'])
test = pd.merge(test, building_info, on=['num'], how='inner')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            10080 non-null  int64  
 1   date_time      10080 non-null  object 
 2   temperature    3360 non-null   float64
 3   windspeed      3360 non-null   float64
 4   humidity       3360 non-null   float64
 5   precipitation  1680 non-null   float64
 6   insolation     3360 non-null   float64
 7   is_cool        10080 non-null  float64
 8   is_solar       10080 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 787.5+ KB


In [49]:
# 기상 정보 (기온, 풍속, 습도, 강수량, 일조량)
test[['temperature', 'windspeed', 'humidity', 'precipitation', 'insolation']] = test[['temperature', 'windspeed', 'humidity', 'precipitation', 'insolation']].interpolate(method='linear')
test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            10080 non-null  int64  
 1   date_time      10080 non-null  object 
 2   temperature    10080 non-null  float64
 3   windspeed      10080 non-null  float64
 4   humidity       10080 non-null  float64
 5   precipitation  10080 non-null  float64
 6   insolation     10080 non-null  float64
 7   is_cool        10080 non-null  float64
 8   is_solar       10080 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 787.5+ KB


#### feature engineering

- 파생 변수를 생성!


- Categorical Feature Encoding


- Time feature [& Lag feature(이전 시간대의 target value)]

> year, **month**, **day**, weekday/weekend, **hour**, minute, second

In [50]:
train.to_csv(file_path+'mission_train.csv')
test.to_csv(file_path+'mission_test.csv')

### 4. 학습 데이터 분할

In [45]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

NameError: ignored

### 5. 학습 및 평가

In [ ]:
# Random Forest

In [ ]:
print("\nFitting RandomForest...")
model.fit(X_train, y_train)

In [ ]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

### 6. Hyper-parameter Tuning

> GridSearchCV

- 주어진 hyper-parameter의 모든 조합을 테스트하는 모델.

In [ ]:
# GridSearchCV를 이용하여 가장 좋은 성능을 가지는 모델을 찾아봅시다.

param_grid = {

}

scorer =
gcv =

gcv.fit(X_train, y_train)
print("Best Estimator : ", gcv.best_estimator_) # best trained model
print("Best Score : ", -gcv.best_score_)

In [ ]:
# feature importances
best_model = gcv.best_estimator_
y_ = best_model.feature_importances_
x_ = X.columns
plt.figure(figsize=(12, 6))
plt.xticks(rotation=45)
sns.barplot(x=x_, y=y_)
plt.show()

### 7. 테스트 및 제출 파일 생성

In [ ]:
## X_test 만들기
X_test =


In [ ]:
best_params =

best_model = RandomForestRegressor(**best_params)

best_model.fit(X_train, y_train)
preds = best_model.predict(X_test)
preds

In [ ]:
submission =
submission

In [ ]:
submission.to_csv("submission.csv", index=False)